## Load Earth System Science Data (ESSD) datasets

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Load Credentials

In [1]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import sys

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

# import sys
# sys.path.append('../src/')

In [2]:
# import re
import pandas as pd
import numpy as np

import openpyxl

import pint
import pint_pandas
import iam_units
from openscm_units import unit_registry
pint_pandas.PintType.ureg = unit_registry
ureg = unit_registry

In [3]:
import osc_ingest_trino as osc
# import pyarrow as pa
# import pyarrow.parquet as pq
# import json
import io
import uuid

Create an S3 resource for the bucket holding source data

In [4]:
import boto3
s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [5]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

In [6]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'
essd_table_prefix = 'essd_'

In [7]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('default',)
('demo_dv',)
('iceberg_demo',)
('information_schema',)
('pcaf_sovereign_footprint',)
('sandbox',)


In [8]:
def create_essd_table(tablename, df, partitioning=''):
    ingest_table = f'{essd_table_prefix}{tablename}'
    columnschema = osc.create_table_schema_pairs(df, typemap={"datetime64[ns]":"timestamp(6)"})

    drop_table = engine.execute(f"drop table if exists {ingest_schema}.{ingest_table}")
    drop_table.fetchall()

    tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
{'partitioning = ' if partitioning else ''}{partitioning}{',' if partitioning else ''}
format = 'ORC'
)
"""
    print(tabledef)
    qres = engine.execute(tabledef)
    print(qres.fetchall())
    df.to_sql(ingest_table,
              con=engine, schema=ingest_schema, if_exists='append',
              index=False,
              method=osc.TrinoBatchInsert(batch_size = 10000, verbose = True))

Load ESSD data file using pandas *read_excel* and using *ingest_uuid* as the global UUID for this ingestion

In [9]:
# Much better to use disk than memory for large datasets (at least as far as notebook resources are concerned)!
s3_essd_file = bObj = bucket.Object('ESSD/essd_ghg_data.xlsx')
essd_tmpfile = f'/tmp/essd-data.xlsx'
s3_essd_file.download_file(essd_tmpfile)
timestamp = bObj.last_modified.isoformat()

# It takes ~90 seconds (!) to load nearly 600K rows of data
df = pd.read_excel(essd_tmpfile, sheet_name='data',
                   # nrows=10,
                   converters={'year': lambda x: pd.to_datetime(x, format='%Y')},
                   dtype={'gwp100_ar5':'int32', 'value':'float64'},
                   engine='openpyxl')
df['value_units'] = 't CO2e'

In [10]:
# Put year at the end to make for more friendly partitioning
year_index = df.columns.get_loc('year')
essd_data_df = df[df.columns[:year_index].to_list()
                  + df.columns[year_index+1:].to_list()
                  + [df.columns[year_index]]]
essd_data_df = essd_data_df.convert_dtypes()

display(essd_data_df.columns)

Index(['ISO', 'country', 'region_ar6_6', 'region_ar6_10', 'region_ar6_22',
       'region_ar6_dev', 'sector_title', 'subsector_title', 'gas',
       'gwp100_ar5', 'value', 'value_units', 'year'],
      dtype='object')

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [11]:
essd_content_df = pd.read_excel(essd_tmpfile, sheet_name='info', header=None).dropna(axis=0).set_index(0).T
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Dataset',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Data description']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

essd_metadata_df = pd.read_excel(essd_tmpfile, sheet_name='metadata')
custom_meta_fields = { d['Variable']: {k:v for k,v in d.items() if k!='Variable' and v==v} for d in essd_metadata_df.to_dict('records') }

The schemaname for this table is `essd`.  Dunno if it's a good idea or simply redundant to prefix the tablename with `essd_`

In [12]:
essd_data_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568341 entries, 0 to 568340
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   ISO              568341 non-null  string        
 1   country          568341 non-null  string        
 2   region_ar6_6     568341 non-null  string        
 3   region_ar6_10    568341 non-null  string        
 4   region_ar6_22    568341 non-null  string        
 5   region_ar6_dev   568341 non-null  string        
 6   sector_title     568341 non-null  string        
 7   subsector_title  568341 non-null  string        
 8   gas              568341 non-null  string        
 9   gwp100_ar5       568341 non-null  Int32         
 10  value            568341 non-null  Float64       
 11  value_units      568341 non-null  string        
 12  year             568341 non-null  datetime64[ns]
dtypes: Float64(1), Int32(1), datetime64[ns](1), string(10)
memory usage: 55.3 

In [13]:
create_essd_table('ghg_data', essd_data_df, "array['year']")
del(essd_data_df)


create table if not exists osc_datacommons_dev.sandbox.essd_ghg_data(
    ISO varchar,
    country varchar,
    region_ar6_6 varchar,
    region_ar6_10 varchar,
    region_ar6_22 varchar,
    region_ar6_dev varchar,
    sector_title varchar,
    subsector_title varchar,
    gas varchar,
    gwp100_ar5 integer,
    value double,
    value_units varchar,
    year timestamp(6)
) with (
partitioning = array['year'],
format = 'ORC'
)

[(True,)]
constructed fully qualified table name as: "sandbox.essd_ghg_data"
inserting 10000 records
  ('ABW', 'Aruba', 'Latin America and Caribbean', 'Latin America and Caribbean', 'Caribbean', 'developing', 'Buildings', 'Residential', 'CO2', 1, 545.2099, 't CO2e', TIMESTAMP '1970-01-01 00:00:00')
  ('ABW', 'Aruba', 'Latin America and Caribbean', 'Latin America and Caribbean', 'Caribbean', 'developing', 'Buildings', 'Residential', 'CH4', 28, 0.16094121, 't CO2e', TIMESTAMP '1970-01-01 00:00:00')
  ('ABW', 'Aruba', 'Latin America and Caribbean', 'Latin Americ

Verify that we can restore data and metadata

Grab Sector, Region, 100yr GWPs, and GH4_gwps from one of the two main ESSD data tables (they are the same in both).

In [14]:
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Sector Classification',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Sectors']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = None

In [15]:
essd_sectors_df = pd.read_excel(essd_tmpfile, sheet_name='sector_classification',
                                # nrows=10,
                                engine='openpyxl').convert_dtypes()
display(essd_sectors_df.columns)

create_essd_table('sectors', essd_sectors_df)

Index(['EDGAR_code', 'fossil_bio', 'sector_title', 'description',
       'subsector_title', 'IPCC_2006'],
      dtype='object')


create table if not exists osc_datacommons_dev.sandbox.essd_sectors(
    EDGAR_code varchar,
    fossil_bio varchar,
    sector_title varchar,
    description varchar,
    subsector_title varchar,
    IPCC_2006 varchar
) with (

format = 'ORC'
)

[(True,)]
constructed fully qualified table name as: "sandbox.essd_sectors"
inserting 208 records
  ('1A1a1', 'bio', 'Energy systems', 'Public Electricity Generation (biomass)', 'Electricity & heat', '1.A.1.a.i')
  ('1A1a1', 'fossil', 'Energy systems', 'Public Electricity Generation ', 'Electricity & heat', '1.A.1.a.i')
  ('1A1a2', 'bio', 'Energy systems', 'Public Combined Heat and Power gen. (biom.)', 'Electricity & heat', '1.A.1.a.ii')
  ...
  ('7C2', 'fossil', 'Industry', 'Indirect N2O from NH3 emitted in cat. 2-3', 'Other (industry)', '5.A')
batch insert result: [(208,)]


In [16]:
essd_regions_df = pd.read_excel(essd_tmpfile, sheet_name='region_classification',
                                # nrows=10,
                                engine='openpyxl').convert_dtypes()
display(essd_regions_df.columns)

create_essd_table('regions', essd_regions_df)

Index(['ISO', 'name', 'region_ar6_6', 'region_ar6_10', 'region_ar6_22',
       'region_ar6_dev'],
      dtype='object')


create table if not exists osc_datacommons_dev.sandbox.essd_regions(
    ISO varchar,
    name varchar,
    region_ar6_6 varchar,
    region_ar6_10 varchar,
    region_ar6_22 varchar,
    region_ar6_dev varchar
) with (

format = 'ORC'
)

[(True,)]
constructed fully qualified table name as: "sandbox.essd_regions"
inserting 250 records
  ('AFG', 'Afghanistan', 'Asia and developing Pacific', 'Southern Asia', 'Rest of Southern Asia', 'ldc')
  ('ALA', 'Åland Islands', 'Developed Countries', 'Europe', 'Northern and western Europe', 'developed')
  ('ALB', 'Albania', 'Developed Countries', 'Europe', 'Southern and eastern Europe', 'developed')
  ...
  ('SCG', 'Serbia and Montenegro', 'Eastern Europe and West-Central Asia', 'Eurasia', 'Eurasia', 'developed')
batch insert result: [(250,)]


In [17]:
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Region Classification',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Regions']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = None

Now deal with the gas species.  We'll annotate our dataframe with PINT units

In [18]:
essd_gwp_df = pd.read_excel(essd_tmpfile, sheet_name='100_yr_gwps',
                            # nrows=10,
                            dtype={'gwp_ar5':'int32'}, engine='openpyxl')

display(essd_gwp_df.columns)

Index(['gas', 'gwp_ar5'], dtype='object')

In [19]:
ureg.define("CH4_Combustion = nan CH4")
ureg.define("CH4_Fugitive = nan CH4")
ureg.define("CH4_Process = nan CH4")

ch4_dict = {
    'CH4 Biogenic': 'CH4',
    'CH4 Fossil (Combustion)': 'CH4_Combustion',
    'CH4 Fossil (Fugitive)': 'CH4_Fugitive',
    'CH4 Fossil (Process)': 'CH4_Process'
}

def convert_gas_to_pint_species(s):
    """For a series S of gas species, return the species name known to pint"""
    return [ch4_dict[g] if g in ch4_dict else g.replace('c-', 'C').replace('-','') for g in s.tolist()]

essd_gwp_df = essd_gwp_df.assign(units=lambda x: convert_gas_to_pint_species(x.gas)).convert_dtypes()

In [20]:
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Gas Species',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Regions']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = None

create_essd_table('gwp_100yr', essd_gwp_df)


create table if not exists osc_datacommons_dev.sandbox.essd_gwp_100yr(
    gas varchar,
    gwp_ar5 integer,
    units varchar
) with (

format = 'ORC'
)

[(True,)]
constructed fully qualified table name as: "sandbox.essd_gwp_100yr"
inserting 33 records
  ('C2F6', 11100, 'C2F6')
  ('C3F8', 8900, 'C3F8')
  ('C4F10', 9200, 'C4F10')
  ...
  ('SF6', 23500, 'SF6')
batch insert result: [(33,)]


In [21]:
essd_ch4_df = pd.read_excel(essd_tmpfile, sheet_name='CH4_gwps',
                            # nrows=10,
                            dtype={'gwp_ar5':'int32', 'subsector':'str'}, engine='openpyxl').convert_dtypes()

display(essd_ch4_df.columns)

Index(['sector_code', 'fossil_bio', 'gas', 'gwp_ar5_feedbacks', 'gwp_ar5',
       'description', 'subsector', 'chapter_title', 'subsector_title'],
      dtype='object')

In [22]:
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) CH4 GWPs',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Regions']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = None

create_essd_table('ch4_gwp', essd_ch4_df)


create table if not exists osc_datacommons_dev.sandbox.essd_ch4_gwp(
    sector_code varchar,
    fossil_bio varchar,
    gas varchar,
    gwp_ar5_feedbacks bigint,
    gwp_ar5 integer,
    description varchar,
    subsector varchar,
    chapter_title varchar,
    subsector_title varchar
) with (

format = 'ORC'
)

[(True,)]
constructed fully qualified table name as: "sandbox.essd_ch4_gwp"
inserting 122 records
  ('1A1a1', 'bio', 'CH4 Biogenic', 34, 28, 'Public Electricity Generation', '6.2', 'Energy systems', 'Biomass energy systems')
  ('1A1a1', 'fossil', 'CH4 Fossil (Combustion)', 34, 28, 'Public Electricity Generation', '6.1', 'Energy systems', 'Electricity & heat')
  ('1A1a2', 'bio', 'CH4 Biogenic', 34, 28, 'Public Combined Heat and Power gen.', '6.2', 'Energy systems', 'Biomass energy systems')
  ...
  ('4E', 'bio', 'CH4 Biogenic', 34, 28, 'Savannah fires', '7.6', 'AFOLU', 'Biomass burning (CO2, CH4)')
batch insert result: [(122,)]


In [23]:
s3_gwp100_file = bObj = bucket.Object('ESSD/essd_ghg_data_gwp100.xlsx')
essd_gwp100_tmpfile = f'/tmp/essd-gwp100.xlsx'
s3_gwp100_file.download_file(essd_gwp100_tmpfile)
timestamp = bObj.last_modified.isoformat()

# This takes about 30sec to execute
df = pd.read_excel(essd_gwp100_tmpfile, sheet_name='data',
                   # nrows=10,
                   converters={'year': lambda x: pd.to_datetime(x, format='%Y')},
                   dtype={'gwp100_ar5':'int32',
                          'CO2':'float64','CH4':'float64','N2O':'float64','Fgas':'float64','GHG':'float64'},
                   engine='openpyxl')
for col in ['CO2','CH4','N2O','Fgas','GHG']:
    df.insert(df.columns.get_loc(col)+1, col + "_units", "t CO2" if col=='CO2' else "t CO2e")

In [27]:
# Put year at the end to make for more friendly partitioning
year_index = df.columns.get_loc('year')
essd_gwp100_df = df[df.columns[:year_index].to_list()
                    + df.columns[year_index+1:].to_list()
                    + [df.columns[year_index]]]
essd_gwp100_df = essd_gwp100_df.convert_dtypes()
display(essd_gwp100_df.columns)

Index(['ISO', 'country', 'region_ar6_6', 'region_ar6_10', 'region_ar6_22',
       'region_ar6_dev', 'sector_title', 'subsector_title', 'CO2', 'CO2_units',
       'CH4', 'CH4_units', 'N2O', 'N2O_units', 'Fgas', 'Fgas_units', 'GHG',
       'GHG_units', 'year'],
      dtype='object')

In [24]:
essd_gwp100_content_df = pd.read_excel(essd_gwp100_tmpfile, sheet_name='info', header=None).dropna(axis=0).set_index(0).T
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) GWP100 Dataset',
    'author': essd_content_df[['Author & contact']].squeeze(),
    'contact': essd_content_df[['Author & contact']].squeeze(),
    'description': essd_content_df[['Data description']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

essd_gwp100_metadata_df = pd.read_excel(essd_gwp100_tmpfile, sheet_name='metadata')
custom_meta_fields = { d['Variable']: {k:v for k,v in d.items() if k!='Variable' and v==v} for d in essd_metadata_df.to_dict('records') }

In [25]:
custom_meta_content

{'title': 'Earth System Science Data (ESSD) GWP100 Dataset',
 'author': 'William F. Lamb (Lamb@mcc-berlin.net) compiled this data from underlying sources (see metadata).',
 'contact': 'William F. Lamb (Lamb@mcc-berlin.net) compiled this data from underlying sources (see metadata).',
 'description': 'This data file provides global, regional, national, and sectoral estimates of greenhouse gas (GHG) emissions from 1970-2019',
 'release_date': '2021-10-13 14:49:22'}

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [28]:
create_essd_table('gwp100_data', essd_gwp100_df, "array['year']")
del(essd_gwp100_df)


create table if not exists osc_datacommons_dev.sandbox.essd_gwp100_data(
    ISO varchar,
    country varchar,
    region_ar6_6 varchar,
    region_ar6_10 varchar,
    region_ar6_22 varchar,
    region_ar6_dev varchar,
    sector_title varchar,
    subsector_title varchar,
    CO2 double,
    CO2_units varchar,
    CH4 double,
    CH4_units varchar,
    N2O double,
    N2O_units varchar,
    Fgas double,
    Fgas_units varchar,
    GHG double,
    GHG_units varchar,
    year timestamp(6)
) with (
partitioning = array['year'],
format = 'ORC'
)

[(True,)]
constructed fully qualified table name as: "sandbox.essd_gwp100_data"
inserting 10000 records
  ('ABW', 'Aruba', 'Latin America and Caribbean', 'Latin America and Caribbean', 'Caribbean', 'developing', 'Buildings', 'Residential', 545.2099, 't CO2', 4.34541267, 't CO2e', 1.310954463, 't CO2e', 0.0, 't CO2e', 550.866267133, 't CO2e', TIMESTAMP '1970-01-01 00:00:00')
  ('ABW', 'Aruba', 'Latin America and Caribbean', 'Latin America and Car

In [29]:
# This table is not so large that we need to load it via the filesystem

import io

bObj = bucket.Object('ESSD/essd_lulucf_data.xlsx')
lulucf_bytes = io.BytesIO(bucket.Object(bObj.key).get()['Body'].read())
timestamp = bObj.last_modified.isoformat()

df = pd.read_excel(lulucf_bytes, sheet_name='data',
                   # nrows=10,
                   converters={'year': lambda x: pd.to_datetime(x, format='%Y')}, engine='openpyxl')

for col in ['blue', 'houghton', 'oscar', 'mean']:
    df.insert(df.columns.get_loc(col)+1, col + "_units", "t CO2e")

# Put year at the end to make for more friendly partitioning
year_index = df.columns.get_loc('year')
lulucf_df = df[df.columns[:year_index].to_list()
               + df.columns[year_index+1:].to_list()
               + [df.columns[year_index]]]
lulucf_df = lulucf_df.convert_dtypes()

display(lulucf_df.columns)

Index(['region_ar6_6', 'region_ar6_10', 'blue', 'blue_units', 'houghton',
       'houghton_units', 'oscar', 'oscar_units', 'mean', 'mean_units', 'year'],
      dtype='object')

In [30]:
lulucf_content_df = pd.read_excel(lulucf_bytes, sheet_name='info', header=None).dropna(axis=0).set_index(0).T
custom_meta_content = {
    'title': 'Earth System Science Data (ESSD) Land Use Change and Forestry Dataset',
    'author': lulucf_content_df[['Author']].squeeze(),
    'contact': lulucf_content_df[['Contact']].squeeze(),
    'description': lulucf_content_df[['Info']].squeeze(),
    'release_date': essd_content_df[['Last date of compilation']].squeeze(),
    # How should we describe our transformative step here?
}

custom_meta_fields = { 'Units': { k:'tCO2' for k in ['blue', 'houghton', 'oscar', 'mean'] }}

In [31]:
custom_meta_content

{'title': 'Earth System Science Data (ESSD) Land Use Change and Forestry Dataset',
 'author': 'Julia Pongratz',
 'contact': 'William F. Lamb (Lamb@mcc-berlin.net)',
 'description': 'This data file provides historical estimates of CO2 emissions from land use, land use change and forestry. It is based on three land use bookeeping models (BLUE, Houghton & Nassikas, OSCAR). Following a practice established by the global carbon budget, and recently the Working Group I of the IPCC (6th Assessment Report), CO2-LULUCF emissions are taken as the mean of these three models.',
 'release_date': '2021-10-13 14:49:22'}

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [32]:
create_essd_table('lulucf', lulucf_df, "array['year']")
del(lulucf_df)


create table if not exists osc_datacommons_dev.sandbox.essd_lulucf(
    region_ar6_6 varchar,
    region_ar6_10 varchar,
    blue double,
    blue_units varchar,
    houghton double,
    houghton_units varchar,
    oscar double,
    oscar_units varchar,
    mean double,
    mean_units varchar,
    year timestamp(6)
) with (
partitioning = array['year'],
format = 'ORC'
)

[(True,)]
constructed fully qualified table name as: "sandbox.essd_lulucf"
inserting 1700 records
  ('Africa', 'Africa', 247575204.928, 't CO2e', 25404728.72, 't CO2e', 107474280.0, 't CO2e', 126818071.216, 't CO2e', TIMESTAMP '1850-01-01 00:00:00')
  ('Africa', 'Africa', 229989415.568, 't CO2e', 25494548.016, 't CO2e', 106888040.0, 't CO2e', 120790667.861333, 't CO2e', TIMESTAMP '1851-01-01 00:00:00')
  ('Africa', 'Africa', 226338781.84, 't CO2e', 25584817.984, 't CO2e', 106265160.0, 't CO2e', 119396253.274667, 't CO2e', TIMESTAMP '1852-01-01 00:00:00')
  ...
  ('Middle East', 'Middle East', -5453068.912, 't CO2e', 1

## Load metadata following an ingestion process into trino metadata store

### The schema is *metastore*, and the table names are *meta_schema*, *meta_table*, *meta_field*